In [ ]:
# Reload all previous stuff, not sure how to do this without redoing everything...
stateList = ['S', 'I', 'R']
paramList = ['beta', 'gamma']
from pygom import Transition, TransitionType
odeList = [
    Transition(origin='S', equation='-beta*S*I', transition_type=TransitionType.ODE),
    Transition(origin='I',equation='beta*S*I - gamma*I', transition_type=TransitionType.ODE),
    Transition(origin='R', equation='gamma*I', transition_type=TransitionType.ODE) 
]
transList = [
    Transition(origin='S', destination='I', equation='beta*S*I', transition_type=TransitionType.T),
    Transition(origin='I', destination='R', equation='gamma*I', transition_type=TransitionType.T)
]

# Solving the model

We will now find deterministic solutions to the SIR model.
First we must import the relevant class

In [ ]:
from pygom import DeterministicOde

Now we initialize the class, which constructs our ODE system from all the information we have provided.
For now, let's use both approaches:

In [ ]:
model = DeterministicOde(stateList, paramList, ode=odeList)
model2 = DeterministicOde(stateList, paramList, transition=transList)

We can verify the model equations are what we'd expect by using the `get_ode_eqn()` function.

In [ ]:
print(model.get_ode_eqn())
print(model2.get_ode_eqn())

where we can see that building the model via equations or transitions results in the same equations corresponding to their respective $S$, $I$ and $R$ state.
From now on, we proceed with just `model`, safe in the knowledge that they are the same.

```{tip}
In addition to showing the equation form of the ODEs, we can also display them as either symbols or latex code, which can save some extra typing when porting the equations to another document.
```

In [ ]:
model.print_ode()

In [ ]:
model.print_ode(True)

<!-- Here the SIR model was provided to PyGOM as a set ODEs by using the {class}`.Transition` to define them.  
We have also provided the capability to obtain a *best guess* transition matrix when only the ODEs are available. See the section {doc}`unrollOde` for more information, and in particular {doc}`unrollSimple` for the continuing demonstration of the SIR model. -->

<!-- ## Extracting model information

We may wish to determine if the set of ODEs are linear. 
model.linear_ode()
Since we know that the SIR model is not linear, we may want to have a look at the Jacobian.

model.get_jacobian_eqn()
Or maybe we want to know the gradient.
model.get_grad_eqn()
```{Warning}
Invoking the functions that compute the derivatives, $f(x)$, `model.ode()` or `model.jacobian()`, will return an error

These functions are used to solve the ODEs numerically, and require values of initial state values, time, and parameter values.
```
For setting initial conditions, the numeric values of the states **must** be set in the same order that list of states appear. We can use the following to check the state ordering, as well as displaying all of the states that we have included.
model.state_list
#TODO unsure if this is needed

There is currently no mechanism to set the numeric initial conditions the states when the states are defined. This is because of an implementation issue with external package, such as solving an initial value problem. -->

## Initial value problem

We can calculate the time evolution of the system given the values of the initial conditions and parameters.

1. Define the model parameters. We can call `parameters` to check what is required

In [ ]:
model.parameters

we then pass them to the class via a list of tuples

In [ ]:
paramEval = [('beta',0.5), ('gamma',1.0/3.0)]
model.parameters = paramEval

and can verify that this was successful

In [ ]:
model.parameters

2. Provide initial conditions for the states.

In [ ]:
i0=1e-6
initialState = [1-i0, i0, 0]

model.ode(state=initialState, t=1)

```{note}
Fractional SIR models are subject to the constraint $S(t)+I(t)+R(t)=1$. It is up to the user to ensure that the initial conditions adhere to any constraints.
```


3. Implement an ODE solver.

We are well equipped to solve an initial value problem, using the standard numerical integrator such as `odeint <scipy.integrate.odeint>` from `scipy.integrate`. We also used `matplotlib.pyplot` for plotting and `linspace <numpy.linspace>` to create the time vector.

In [ ]:
import scipy.integrate
import numpy

t = numpy.linspace(0, 150, 100)

solution = scipy.integrate.odeint(model.ode, initialState, t)

We can plot our solution to observe a standard SIR shape.

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(t, solution[:,0], label='S')
plt.plot(t, solution[:,1], label='I')
plt.plot(t, solution[:,2], label='R')
plt.xlabel('Time')
plt.ylabel('Population proportion')
plt.title('Standard SIR model')
plt.legend(loc=0)
plt.show()

Alternatively, we can integrate and plot via the **ode** object which we initialized earlier.

In [ ]:
model.initial_values = (initialState, t[0])

model.parameters = paramEval

solution = model.integrate(t[1::])

model.plot()

We could solve the ODEs above using the Jacobian as well. Unfortunately, it does not help because the number of times the Jacobian was evaluated was zero, as expected given that our set of equations are not stiff.

In [ ]:
#TODO what does this show?
%timeit solution1, output1 = scipy.integrate.odeint(model.ode, initialState, t, full_output=True)


In [ ]:

%timeit solution2, output2 = scipy.integrate.odeint(model.ode, initialState, t, Dfun=model.jacobian, mu=None, ml=None, full_output=True)


In [ ]:

%timeit solution3, output3 = model.integrate(t, full_output=True)

It is important to note that we return our Jacobian as a dense square matrix. Hence, the two argument (mu,ml) for the ODE solver was set to `None` to let it know the output explicitly.

## Solving the forward sensitivity equation

The sensitivity equations are also solved as an initial value problem. Let us redefine the model in the standard SIR order and we solve it with the sensitivity all set at zero, i.e. we do not wish to infer the initial value of the states.

In [ ]:
stateList = ['S', 'I', 'R']

model = DeterministicOde(stateList, paramList, ode=odeList)

initialState = [1, 1.27e-6, 0]

paramEval = [('beta', 0.5), ('gamma', 1.0/3.0)]

model.parameters = paramEval

solution = scipy.integrate.odeint(model.ode_and_sensitivity, numpy.append(initialState, numpy.zeros(6)), t)

In [ ]:
{
    "tags": [
        "hide-input",
    ]
}
f,axarr = plt.subplots(3,3);

f.text(0.5,0.975,'SIR with forward sensitivity solved via ode',fontsize=16,horizontalalignment='center',verticalalignment='top')

axarr[0,0].plot(t, solution[:,0])

axarr[0,0].set_title('S')

axarr[0,1].plot(t, solution[:,1])

axarr[0,1].set_title('I')

axarr[0,2].plot(t, solution[:,2]);

axarr[0,2].set_title('R')

axarr[1,0].plot(t, solution[:,3])

axarr[1,0].set_title(r'state S parameter $beta$')

axarr[2,0].plot(t, solution[:,4])

axarr[2,0].set_title(r'state S parameter $gamma$')

axarr[1,1].plot(t, solution[:,5])

axarr[1,1].set_title(r'state I parameter $beta$')

axarr[2,1].plot(t, solution[:,6])

axarr[2,1].set_title(r'state I parameter $gamma$')

axarr[1,2].plot(t, solution[:,7])

axarr[1,2].set_title(r'state R parameter $beta$')

axarr[2,2].plot(t, solution[:,8])

axarr[2,2].set_title(r'state R parameter $gamma$')

plt.tight_layout()

plt.show()

This concludes the introductory example and we will be moving on to look at parameter estimation next in {doc}`estimate1` and the most important part in terms of setting up the ODE object; defining the equations in various different ways in {doc}`transition`.